# clustering of GPS coordinates via  M-way tree

In [2]:
import numpy as np, pandas as pd
import json
import time
import bisect
from util import *

/home/chandu/ckm/py38/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# # Show part of the data
# data = pd.read_csv("../data/usergps.csv", delimiter="\t", header=None)
# data = data.iloc[:,:4]

In [3]:
# print(data.head())

In [4]:
# # data = data.iloc[:1000,:]
# # print(data.shape)
# """
# Convert degree decimal (DD) to degree, minute, and second format (DMS) 
# """

# data=data.applymap(dd2dms)
# print(data.head())

In [5]:
# print(data.shape)

# Create multilevel hash map

# We want to build a tree like below
![caption](tree.jpg)

In [6]:
contact_dist = 5

# Create trees

In [7]:
# lattree = CreateGPSTree('lat',contact_dist)

# with open("../data/lat_tree_before.json","w") as fid:
#     fid.write(json.dumps(lattree, indent=5))
# with open("../data/lat_tree_after.json","w") as fid:
#     fid.write(json.dumps(lattree, indent=5))
    
# longtree = CreateGPSTree('long',contact_dist)
# with open("../data/long_tree_before.json","w") as fid:
#     fid.write(json.dumps(longtree, indent=5))
# with open("../data/long_tree_after.json","w") as fid:
#     fid.write(json.dumps(longtree, indent=5))
    

In [8]:
# # #  delete the tresss for after use in offline mode.
# del lattree
# del longtree

# Data for Testing Static case

In [9]:
# # Generate some fake data
# samples = 20000000
# degree = np.random.randint(7,39, samples)
# minutes = np.random.randint(0,59, samples)
# col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
# lat1 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# # degree = np.random.randint(7,39, samples)
# # minutes = np.random.randint(0,59, samples)
# col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
# lat2 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# degree = np.random.randint(66,99, samples)
# minutes = np.random.randint(0,59, samples)
# col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
# long1 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# # degree = np.random.randint(66,99, samples)
# # minutes = np.random.randint(0,59, samples)
# col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
# long2 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# data=pd.concat([lat1, long1, lat2, long2], axis=1)
# print(data.head())

In [8]:
samples = 20000000
rows =  np.arange(0,samples)
degree = np.random.randint(7,39, samples)
minutes = np.random.randint(0,59, samples)
col1 = np.random.rand(samples)
col2 = np.random.uniform(0,59, samples)
lat1 = pd.DataFrame({"0":[(r,d,m,s+f) for r,d, m, s, f in zip(rows,degree, minutes,col1, col2)]})

print(lat1.head())
lat1.to_csv(r'pyafter.txt', header=False, index=False)

                                 0
0   (0, 18, 51, 54.29690177001075)
1  (1, 17, 31, 55.594497560544745)
2    (2, 7, 50, 20.87726809400921)
3   (3, 32, 6, 39.609994994723166)
4   (4, 26, 47, 20.24812753988229)


In [6]:
d=pd.read_csv('pybefore.csv',header=None)
print(d.head())

                                0
0  (0, 29, 44, 5.737904899786317)
1  (1, 28, 37, 41.24992928133539)
2  (2, 34, 15, 55.15784171878894)
3  (3, 11, 33, 52.46618081640381)
4   (4, 25, 6, 4.347807177006583)


# Data for testing dynamic case

In [10]:
# Generate some fake data
samples = 20000000
degree = np.random.randint(7,39, samples)
minutes = np.random.randint(0,59, samples)
col1 = np.random.rand(samples)
col2 = np.random.uniform(0,59, samples)
lat1 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# degree = np.random.randint(7,39, samples)
# minutes = np.random.randint(0,59, samples)
col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
lat2 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
degree = np.random.randint(66,99, samples)
minutes = np.random.randint(0,59, samples)
col1 = np.random.rand(samples)
col2 = np.random.uniform(0,59, samples)
long1 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
# degree = np.random.randint(66,99, samples)
# minutes = np.random.randint(0,59, samples)
col1 = np.random.rand(samples)
# col2 = np.random.uniform(0,59, samples)
long2 = pd.DataFrame({"0":[(d,m,s+f) for d, m, s, f in zip(degree, minutes,col1, col2)]})
data=pd.concat([lat1, long1, lat2, long2], axis=1)
print(data.head())

                              0                             0  \
0    (9, 39, 47.04237522957328)   (87, 7, 28.391602397776253)   
1  (32, 43, 50.675141662914655)   (73, 55, 28.39403478153326)   
2   (28, 1, 14.228885863806806)  (77, 46, 59.097831650368875)   
3   (10, 50, 45.70082223074443)  (83, 19, 13.474401704483482)   
4   (22, 7, 0.1946891674461001)   (84, 22, 9.428105626498168)   

                              0                             0  
0    (9, 39, 47.17617157686021)   (87, 7, 28.008109016592172)  
1   (32, 43, 50.59820391475288)  (73, 55, 28.307190362929028)  
2   (28, 1, 14.119150280788606)   (77, 46, 59.11980223175994)  
3  (10, 50, 46.495978974086334)   (83, 19, 14.04829273287082)  
4   (22, 7, 0.7166350982153299)   (84, 22, 9.060658184093095)  


In [11]:
# #  sample Data for testing static/dynamic case
# data = pd.read_csv("../data/sample.csv")
# data  = data.iloc[:,1:5]
# data = data.applymap(eval)
# print(data)


In [12]:
# list(enumerate(data.iloc[:,0]))

In [13]:
# Load tress

with open("../data/lat_tree_before.json","r") as fid:
    lat_tree_before = json.load(fid)
with open("../data/lat_tree_after.json","r") as fid:
    lat_tree_after = json.load(fid)
    
# with open("../data/long_tree_before.json","r") as fid:
#     long_tree_before = json.load(fid)
# with open("../data/long_tree_after.json","r") as fid:
#     long_tree_after = json.load(fid)

# Static Case

In [14]:
# # #  map data to trees
# start = time.perf_counter()
# MapGPS(lat_tree_before, data.iloc[:,0], contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)
# start = time.perf_counter()
# MapGPS(lat_tree_after, data.iloc[:,2], contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)

# #  Intersect trees

# start = time.perf_counter()
# contact_list = IntersectGPSTree(lat_tree_before, lat_tree_after, 'lat', contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)

In [15]:
# contact_list

In [16]:
# #  map data to trees
# start = time.perf_counter()
# MapGPS(long_tree_before, data.iloc[:,1], contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)
# start = time.perf_counter()
# MapGPS(long_tree_after, data.iloc[:,3], contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)

# #  Intersect trees

# start = time.perf_counter()
# contact_list = IntersectGPSTree(long_tree_before, long_tree_after, 'long', contact_dist)
# end = time.perf_counter()
# print("Elapsed time: ", end-start)

In [17]:
# contact_list

# Dynamic Case

In [18]:
#  map data to trees
start = time.perf_counter()
MapGPS(lat_tree_before, data.iloc[:,0], contact_dist)
end = time.perf_counter()
print("Elapsed time for mapping to tree: ", end-start)
start = time.perf_counter()
MapGPS(lat_tree_after, data.iloc[:,2], contact_dist)
end = time.perf_counter()
print("Elapsed time for mapping to tree: ", end-start)

#  Intersect trees

start = time.perf_counter()
contact_list = FastPedestrianIntersectGPSTree(lat_tree_before, lat_tree_after, 'lat', contact_dist)
end = time.perf_counter()
print("Elapsed time for Intersection: ", end-start)

Elapsed time for mapping to tree:  205.09433840199995
Elapsed time for mapping to tree:  202.2269994889998
Elapsed time for Intersection:  369.76297627200006


In [19]:
contact_list[0]

{3238663, 15456344}

# Parallel Implementation of the mapping

In [20]:

# data=[1,2,3,1,3,1]
# # sharedtree= manager.dict({"1":manager.list(),"2":manager.list(),"3":manager.list()})
# sharedtree = manager.dict({"1":[],"2":[],"3":[]})
# with Pool(processes=3) as pool:
#     pool.map(partial(mapTo, tree=sharedtree), list(enumerate(data,start=1)))
    

In [21]:
# def mapTo(i_d,tree):
#     idx,item = i_d
#     l = tree[str(item)]
#     l.append(idx)
#     tree[str(item)] = l
    

       

In [22]:
# from functools import partial
# from multiprocessing import Pool, Manager
# manager = Manager()
# lat_shared_before = manager.dict(lat_tree_before)

# start = time.perf_counter()
# with Pool(processes=8) as pool:
#     pool.map(partial(MapGPSInParallel, tree=lat_shared_before, contact_dist=contact_dist),list(enumerate(data.iloc[:,0])))

# # MapGPS(lat_tree_before, data.iloc[:,0], contact_dist)

# end = time.perf_counter()
# print("Elapsed time: ", end-start)

In [23]:
# from objsize import get_deep_size
# print(get_deep_size(tree))
# print(get_deep_size(gps_tree))